# K - Nearest Neighbor Algorithm (KNN)

Nearest Neighbor means that we can analize observations that are the most similar to others in a dataset, and classify them based on the values that are the closest point

K is what allows the user to select the size / number of neighbors that we take into consideration per point.


### Pseudocode

Function euclidean_distance:
- calculate the Euclidean distance between two vectors
- Return square root of distance

Function get_neighbors:
- Initialize an empty list distances
- For each row in train:
    * Calculate Euclidean distance between test_row and the current row
    * Append (current row, distance) to distances list
- Sort distances based on distance in ascending order
- Return the first num_neighbors rows from distances

Function predict_classification:
- Get the K nearest neighbors
- Extract output values from neighbors
- Return the mode (most frequent value) of the output values

Test distance function:
- Initialize the dataset with sample data
- Call predict_classification with the dataset, the first row of the dataset, and 3 as arguments, store the result in prediction
- Print expected and predicted class


### Example

In [ ]:
# Importing
import csv
from math import sqrt
import numpy as np

In [ ]:
def load_csv(filename):
    dataset = []
    columns_to_remove = [0,1,2,3,4]  # Indices of columns to remove (0-based indexing)

    with open(filename, 'r', encoding='cp437') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if not row:
                continue

            # Remove specified columns
            modified_row = [row[i] for i in range(len(row)) if i not in columns_to_remove]
            dataset.append(modified_row)

    if len(dataset) > 0:
        dataset = dataset[1:]  # Exclude the first row
    else:
        print("Dataset is empty or does not contain any rows.")

    return dataset

Here we take a column for the current row, then we remove any whitespace using the function strip(), and convert the string to float.

We need to do this because we need to calculate the distances between datapoints, so by converting strings of numbers to actual numerical values, we can actually work with them.

In [ ]:
def str_column_to_float(dataset, column):
    for row in dataset:
        try:
            row[column] = float(row[column].strip())
        except ValueError:
            row[column] = np.nan

We create a "dictionary" called lookup where we iterate through the unique class values, assign an integer label to each unique value, and print them.

In [ ]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values) # Converts the list of class values to a set to obtain unique class values.
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
        print('[%s] => %d' % (value, i))
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

We determine the range of values for each column in the dataset.

In [ ]:
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list() #list to store the minmax values for each column.
    for i in range(len(dataset[0])): # Iterates through each column
        col_values = [row[i] for row in dataset] # Extracts the values from the current column
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

In [ ]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)): # Iterates through each row in the dataset and then through each column within that row.
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

The first step for KNN is to calculate the distance between two rows in a dataset.

In [ ]:
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1) - 1):
        distance += (row1[i] - row2[i]) ** 2
    return sqrt(distance)

After that, we need to calculate the distance between each observation in the dataset to the new piece of data. Then classify all observations in the training dataset by their distance to the new data.

We select the top *k* to return as the most similar neighbors.

Here, lambda defines an anonymous function that takes a single tuple and returns the second element of the tuple, *tup[1]*. The data point and its distance from the test_row are added as a tuple to the distances list.

In [ ]:
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
    distances = list() # Initialize an Empty List for Distances
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

After identifying the classes of the K nearest neighbors for a given data point, we have a list of output values representing the classes of these neighbors.

The max() function is then applied to this list of output values to return the maximum value in that list (in this case, the classes).

**train:** The training dataset containing labeled data points.

**test_row:** The data point for the prediction.

**num_neighbors:** The number of nearest neighbors to consider for the prediction.

In [ ]:
# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count) # finds the most common output label
    return prediction

Main Code

In [ ]:
filename = './datasets/Indicadores_municipales.csv'
dataset = load_csv(filename)

for i in range(len(dataset[0])-1): # iterate through each column excluding the target variable
    str_column_to_float(dataset, i) # convert string to float.

# Convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# Define Neighbors
num_neighbors = 5
# Define a new observation
row = [5.7, 2.9, 4.2, 1.3]

# Predict the label
label = predict_classification(dataset, row, num_neighbors)
print('Data=%s, Predicted: %s' % (row, label))

[Medio] => 0
[Bajo] => 1
[Muy bajo] => 2
[Muy alto] => 3
[Alto] => 4
Data=[5.7, 2.9, 4.2, 1.3], Predicted: 0


### Loss and Optimization Functions

A loss function is used to measure the accuracy of a model’s predictions. It calculates the difference between the predicted output and the actual output for each training sample.

KNN does not inherently involve a loss function in the same way as supervised learning models. No function is fitted to the data, as such, no optimization is performed

However we can still calculate the accuracy of our predictions through cross-validation

In [ ]:
# Importing
from random import seed
from random import randrange

"Folds" refers to the subdivisions into which a dataset is divided for evaluating

For each fold, we iterate n times. Within each iteration, we create a new fold and populate it with fold_size samples randomly selected from the dataset copy.

We use randrange to select a random index from dataset_copy, and add it to the fold. We then remove it from dataset_copy to avoid duplication.

Finally, the fold is added to dataset_split.

In [ ]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds) # Calculates the size of each fold
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy)) # select a random index from dataset_copy
            fold.append(dataset_copy.pop(index)) # remove from dataset_copy to avoid duplication.
        dataset_split.append(fold)
    return dataset_split

Predict the labels for the test dataset using the KNN algorithm

In [ ]:
# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors)
        predictions.append(output)
    return predictions

Comparing the predicted labels with the actual labels

In [ ]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0 # keep track of correct predictions
    for i in range(len(actual)):
        if actual[i] == predicted[i]: #  check if the actual label matches the predicted label
            correct += 1
    return correct / float(len(actual)) * 100.0 # get percentage

Here we apply Cross-validation, which is when the dataset is randomly split up into 'k' groups. One of the groups is used as the test set and the rest are used as the training set.

In [ ]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds) # create a training set by combining all folds except the current fold.
        train_set.remove(fold)
        train_set = sum(train_set, []) # separate the current fold as the test set.
        test_set = list()
        for row in fold: # preparing the test set by creating a copy of each row in the fold
            row_copy = list(row)
            test_set.append(row_copy) # Add copied row to the test_set
            row_copy[-1] = None # remove the label from the row
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold] # Extract the actual labels from the current fold.
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

KNN itself doesn't have an usual training phase with model parameters, but cross-validation helps in understanding how well the algorithm performs.

In [ ]:
# Test the kNN on the penguins dataset
seed(1)
filename = './datasets/Indicadores_municipales.csv'

dataset = load_csv(filename)

# Convert columns to appropriate data types
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)

# Convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

# Evaluate the algorithm
n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)

print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

[Medio] => 0
[Bajo] => 1
[Muy bajo] => 2
[Muy alto] => 3
[Alto] => 4
Scores: [23.625254582484725, 33.604887983706725, 29.735234215885946, 34.623217922606926, 32.17922606924644]
Mean Accuracy: 30.754%


html


In [ ]:
%%shell
jupyter nbconvert --to html /content/KNN.ipynb

[NbConvertApp] Converting notebook /content/KNN.ipynb to html
[NbConvertApp] Writing 624987 bytes to /content/KNN.html
